In [1]:
#------------------------------------------------------------------------------------------------
# HDBSCAN Clustering
# - 문장 임베딩 유사성을 기반으로 유사한 문장을 밀도(밀집도)로 클러스터링하는 HDBSCAN 알고리즘을 이용하는 클러스터링 예.
# - 테스트 data : newstrust_20210601_samlple.json 사용
#
# 참고 소스 : https://https://github.com/KPFBERT/kpfSBERT_Clustering/blob/main/kpfSBERT_clustering.ipynb
# HDBSCAN(Hierarchical Density-Based Spatial Clustering of Applications with Noise) 및 UMAP 설치
# - pip install hdbscan (conda install -c conda-forge hdbscan)
# - pip install umap-learn (conda install -c conda-forge umap-learn)
#
#------------------------------------------------------------------------------------------------
import torch
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging
logger = mlogging(loggername="HDBSCAN-cluster", logfilename="../../../log/cluster")
device = GPU_info()
seed_everything(111)

import hdbscan
import umap

logfilepath:../../../log/cluster_2023-05-17.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#-------------------------------------------------------------------------------------
# 1. 임베딩모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
import torch
from myutils import bi_encoder

bi_encoder_path = "bongsoo/kpf-sbert-128d-v1" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls

 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128

word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------


---bi_encoder---------------------------
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 128, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)
Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 


In [3]:
#-------------------------------------------------------------------------------------
# 2. 문서로딩
# - [{"title":"다온플래닛, 인프라 모니터링 서비스 지원","category":"IT 과학"},
#    {"title":"텔레칩스, 러시아 자동차 OEM에 차량용 반도체 공급","category":"경제"},...]
#-------------------------------------------------------------------------------------
DATA_PATH = 'data/newstrust_20210601_samlple.json'
df = pd.read_json(DATA_PATH, encoding='utf-8')
len(df)

12006

In [18]:
# UMAP 차원축소 함수
def umap_process(corpus_embeddings, 
                 n_components:int=5  # 2 에서 100 값(default 2)
                ):
    
    umap_embeddings = umap.UMAP(n_neighbors=15,
                                n_components=n_components,
                                metric='cosine').fit_transform(corpus_embeddings)
    return umap_embeddings

# HDBSCAN 실행
def hdbscan_process(corpus, 
                    corpus_embeddings, 
                    min_cluster_size=15, # 최소 클러스터링 수
                    min_samples=10,      # 최소 샘플링 수 
                    umap=True, 
                    n_components=5, 
                    method='eom'):
    if umap:
        umap_embeddings = umap_process(corpus_embeddings, n_components)
    else:
        umap_embeddings = corpus_embeddings
        
    cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,  # 최소 클러스터링 수
                              min_samples=min_samples,            # 최소 샘플링 수(기본=None) 
                              allow_single_cluster=True,
                              metric='euclidean',
                              core_dist_n_jobs=1,   # knn_data = Parallel(n_jobs=self.n_jobs, max_nbytes=None) in joblib
                              cluster_selection_method=method).fit(umap_embeddings)
    
    docs_df = pd.DataFrame(corpus, columns=["Doc"])
    docs_df['Topic'] = cluster.labels_
    docs_df['Doc_ID'] = range(len(docs_df))
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ''.join})
    
    print(f'hdbscan_process => cluster.labels_:{cluster.labels_} / len(docs_df):{len(docs_df)} / len(docs_per_topic):{len(docs_per_topic)}')   
    print()
    print('docs_per_topic:', docs_per_topic)
    print()
    
    return docs_df, docs_per_topic


In [19]:
# 클러스터링 시작
import re
from datetime import datetime

#------------------------------------------
# param
#------------------------------------------
CLUSTER_MODE = 'title'

USE_UMAP = True               # True=umap 차원축소 사용 
UMAP_N_COMPONENTS = 15        # 차원 축소시 몇 차원으로 축소할지?

HDBSCAN_MIN_CLUSTER_SIZE = 5 # 최소 클러스터링 계수 
HDBSCAN_MIN_SAMPLES = 10     # 샘플링 계수(몇개까지 포함될때 그룹인지 정의: 기본 = None)
#------------------------------------------

tot_df = pd.DataFrame()

start = datetime.now()
print('작업 시작시간:',start)

previous = start
bt_prev = start

print(' processing start... with CLUSTER_MODE:', CLUSTER_MODE)

category = df.category.unique()

# 카테고리 별루 df 만듬.
df_category = []
for categ in category:
    df_category.append(df[df.category==categ])

cnt = 0
rslt = []
topics = []

# for문 돌면서 데이터 만들어서 df에 적용
for idx, dt in enumerate(df_category):
    
    corpus = dt[CLUSTER_MODE].values.tolist()
    
    # '[보통공통된꼭지제목]' 형태를 제거해서 클러스터링시 품질을 높인다.
    for i, cp in enumerate(corpus):
        corpus[i] = re.sub(r'\[(.*?)\]','',cp)
        #print(corpus[:10])
    
    # 임베딩 벡터 구함
    corpus_embeddings = bi_encoder1.encode(corpus, show_progress_bar=True)
    
    docs_df, docs_per_topic = hdbscan_process(corpus, 
                                              corpus_embeddings,
                                              umap=USE_UMAP,       # 차원 축소 사용 유.무
                                              n_components=UMAP_N_COMPONENTS,  # 차원 축소시 몇 차원으로 축소할지?
                                              method='leaf',
                                              min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
                                              min_samples=HDBSCAN_MIN_SAMPLES,  # 샘플링 계수(몇개까지 포함될때 그룹인지 정의: 기본 = None)
                                             )
        
    cnt += len(docs_df)
    
    rslt.append(docs_df)
    topics.append(docs_per_topic)
    dt['cluster'] = docs_df['Topic'].values.tolist()
    tot_df = pd.concat([tot_df, dt])
    
    bt = datetime.now()
    print('문서 카테고리: ', category[idx], ', 문서수 : ', len(docs_df), ', 클러스터링 수:', len(docs_per_topic)-1, ', 소요시간:', bt - bt_prev)
    bt_prev = bt
    
now = datetime.now()
print('총 문서수:', cnt, ', 카테고리 수: ', len(rslt), ', 총소요시간:', now - previous)
previous = bt

# 클러스터 업데이트
df['cluster'] = tot_df['cluster'].astype(str)

end = datetime.now()

print('작업종료시간 : ', end, ' 총 소요시간:', end - start)

작업 시작시간: 2023-05-17 10:19:36.619434
 processing start... with CLUSTER_MODE: title


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

hdbscan_process => cluster.labels_:[-1 -1 -1 -1 -1  8 -1 -1 -1 -1 11 -1 -1  9 -1 -1  7  9 16  4 -1 15 -1 -1
 -1  7 -1 -1 16  3 -1 -1  9 -1 -1 23  7 -1 17 14 22 -1 21  7 -1 -1 -1 -1
  7 -1 23 -1 -1 21  8 -1 -1 18 23  7  3  6 -1 -1 22 -1 -1  9  6 -1  6 -1
  3 20  3 -1 -1 -1 -1 -1 -1 -1 -1 -1  7  3 -1 -1 14 15 17 -1 -1 -1 23  4
 15 -1  2 16 -1 -1  7 -1 16 13 -1 23 -1  3 13 23 19 -1  7  0  5 15  1 -1
 17 -1 -1  7 -1  9 15 -1 -1  9 -1 -1 -1 -1  5 22 11 14  5 12 -1  0 -1 -1
  7  0 -1 21 -1  2  3 -1 -1 -1 -1  3 -1 -1 16 -1 -1  0  9 -1 10 -1  1 -1
  4 -1 23 -1 16 19 -1 14 -1 -1 -1  3 11  2 -1  5  1 -1 -1 19 -1  0  9 -1
  7  9 14  6 11 -1 15 -1 -1  0 -1 10 16 -1  3 -1 -1 -1  3 13 -1 -1 -1  4
 17  7 19 22 -1 -1 -1  3 -1 -1 -1  7 -1 -1 -1  3 -1 -1 -1  1 -1 18 -1 23
 -1 16 -1 -1 -1  2  1 19 -1  4  3 13 -1 22 -1 -1 10 13 -1 10 -1 -1 -1  9
 21 -1  8 -1 19 -1  0 -1 21 -1 -1 13 -1 -1 -1 -1  9 -1  3 17 17 -1 -1 18
  3 -1 -1 -1  2 -1 10 -1 -1  6 16 -1 10  4  4 -1 -1  3 -1  5 -1 -1  5  0
  7 -1  0 -1 10 

Batches:   0%|          | 0/84 [00:00<?, ?it/s]

hdbscan_process => cluster.labels_:[-1 -1 -1 ... -1 -1  6] / len(docs_df):2662 / len(docs_per_topic):73

docs_per_topic:     Topic                                                Doc
0      -1  텔레칩스, 러시아 자동차 OEM에 차량용 반도체 공급롯데하이마트, 무더위 대비 6월...
1       0  특허청, 러시아·멕시코에 해외지식재산센터 신설화이자 3000만명분 제안 받은 대구…...
2       1  갈 수 없는 길인데…AI 내세워 수수료 후려치기AI 기술 적용한 업무 자동화 솔루션...
3       2  하나투어, 명동 티마크호텔 950억원에 이지스에 매각아이유, 130억 원 청담동 최...
4       3  워크데이 2022 회계연도 1분기 총 매출 11억8000만 달러지평주조, 창립 96...
..    ...                                                ...
68     67  KB금융, 2000억원 규모 정책형 뉴딜 인프라펀드 출시KB금융, ‘KB 뉴딜 인프...
69     68  현대百, '환경경영시스템 인증' 획득삼성카드, 환경경영체제 국제인증 'ISO 140...
70     69  쌍용정보통신 “EBS 방송·교육 인터넷서비스 통합 운영 수주” 공공기관 동반 성장 ...
71     70  에너지경제신문포스코, 반도체・디스플레이용 등 산업가스 사업화 추진포스코, 친환경 산...
72     71  정승일 한전 사장 취임..."탄소중립 시대 전력 패러다임 선도하자"“도움 받던 나라...

[73 rows x 2 columns]

문서 카테고리:  경제 , 문서수 :  2662 , 클러스터링 수: 72 , 소요시간: 0:00:12.708620


...

hdbscan_process => cluster.labels_:[-1  5 -1 -1 -1  0 -1 -1  5 -1 -1  4 -1 -1 -1  4  1  3 -1 -1  5  5 -1 -1
  4 -1 -1 -1 -1  3 -1 -1 -1 -1 -1  4  4 -1  4  4  4  0  3  0  1  3 -1  4
  2 -1  1 -1  6  4  4 -1  4  5  4 -1 -1  5 -1  1 -1 -1  4  4  4  4 -1  0
  1  4  5  6 -1  0  1 -1 -1 -1 -1  0  1  0 -1  4 -1  2 -1  6 -1  2 -1  4
  0  4  4  5 -1  2 -1 -1 -1 -1 -1 -1  0  4 -1  4 -1 -1  4  4 -1  1 -1  3
 -1  1  0  0  4  0  2  4  4  4 -1  3  5  4 -1  4  4 -1  4  0  4 -1 -1  0
 -1  3  6  0 -1  2 -1 -1  3 -1  0 -1 -1 -1  4  4  4  4  4  1  1 -1 -1 -1
 -1 -1 -1  0 -1 -1 -1  1 -1  0 -1 -1 -1  3  0 -1 -1 -1 -1 -1  1  0  0  0
  3  5  5 -1 -1  4 -1  3 -1  5  4  4 -1 -1  2  4 -1  0  2 -1  6 -1  0 -1
  6  4 -1  5  4 -1 -1 -1  1  6  1  4  4  4  0 -1  1  3 -1 -1 -1  1 -1 -1
 -1  0  0  1  1 -1  1 -1  5  5  3  2 -1  2 -1  1  1  5  5] / len(docs_df):259 / len(docs_per_topic):8

docs_per_topic:    Topic                                                Doc
0     -1  원자력안전재단, 방사선안전 교육원 확장 이전나청운(전 서울 풍납중 교장)씨 별세런데

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

hdbscan_process => cluster.labels_:[-1  0  1 -1 -1 -1 -1  1 -1  0  1  1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1  0 -1 -1 -1  0  0  1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1
  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
  1 -1  1  0  1  1 -1] / len(docs_df):79 / len(docs_per_topic):3

docs_per_topic:    Topic                                                Doc
0     -1  美 FDA 전 국장 "코로나, 中 연구소 정황 늘어"다정한 안부 건네는 식탁으로의 ...
1      0  문화도시 청주, 곳곳서 기록문화축제 연다2021 제주여행 포스팅 공모전 개최…상생 ...
2      1  2021년 6월의 6·25전쟁 영웅 '백두산함' 6월 2일 수요일 (음력 4월 22...

문서 카테고리:  라이프스타일 , 문서수 :  79 , 클러스터링 수: 2 , 소요시간: 0:00:03.221427


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

hdbscan_process => cluster.labels_:[-1 -1 -1 -1 -1  1  0  5 -1 -1 -1  5  0  1  0  2 -1  7  7  1 -1  5 -1  1
  4 -1  5  1 -1  4  2  1  8  1 -1  3 -1  3  3 -1  8 -1  3  0  5 -1  8  2
 -1  8  5 -1 -1  5  4 -1 -1 -1  1  2 -1 -1 -1 -1 -1  1 -1  1 -1  7  7  2
 -1  0  3 -1  0  6 -1  0 -1  1 -1  1  2  1 -1  1  1 -1 -1 -1  1  2  1 -1
 -1  0 -1 -1  0  5 -1  7 -1  3  5  6  2  6  6  7  5  7  6  6  4  2 -1  8
 -1  1 -1 -1 -1  5 -1  2  5  0  3  4 -1  0 -1  5 -1 -1 -1 -1  3 -1  0 -1
  6  7 -1 -1 -1 -1 -1  7  1  5 -1  7 -1  2  7  7 -1 -1  7  3  5 -1  1 -1
 -1 -1  5 -1 -1 -1  0  4 -1 -1  4  2 -1  4  1  5  3 -1  1  3 -1  1  2 -1
 -1 -1 -1 -1 -1 -1  0  1 -1 -1  1 -1 -1  0 -1  5  1  1 -1  2 -1  6 -1  2
 -1  4  1 -1 -1  0  4  8 -1  3 -1  7 -1 -1 -1 -1 -1  6 -1  2 -1  0 -1  3
  0 -1  0  0 -1  3  5 -1  0  0 -1  5 -1  6 -1  0  8 -1  2  1  3  7 -1  8
  5 -1] / len(docs_df):266 / len(docs_per_topic):10

docs_per_topic:    Topic                                                Doc
0     -1  '풀백 출격' 손흥민 "이게 바로 토트넘 

In [7]:
# 중복된 cluster 계수를 제거하고, 몇개의 cluster가 있는지 확인.
df_cluster = df.drop_duplicates('cluster')
print(f'len(df_cluster):{len(df_cluster)}')

len(df_cluster):120


In [8]:
# 조건식 작성후 클러스터링 확인.
categ = '사회'
condition = (df.category == categ) & (df.cluster == '3')

test = df[condition]
print(len(test))

test.title.values.tolist()

45


["SK렌터카, 제주지점 공항 인근으로 이전...셔틀 거리 '10→2분'",
 '오늘의 운세- 2021년 6월 2일(음력 4월 22일)',
 '오늘의 날씨- 2021년 6월 2일',
 '인물 동정 (6월 2일)',
 '행사 알림(6월 2일)',
 '[행사] 6월 2일 천안시',
 '[동정] 6월 2일 천안시 읍면동장',
 '[오늘의 날씨] 2021년 6월 1일',
 '6·10 만세운동 네 명의 주역 ‘6월의 독립운동가’',
 '[동정]속초 고성 인제 2021년 6월 2일',
 '[동정]춘천 홍천 2021년 6월 2일',
 '[동정]원주 횡성 2021년 6월 2일',
 '[토막소식]동해 삼척 태백 2021년 6월 2일',
 '[동정]동해 삼척 태백 2021년 6월 2일',
 '[동정]영월 평창 정선 2021년 6월2일',
 '[동정]인물 2021년 6월 2일',
 '[토막소식]속초 고성 인제 2021년 6월 2일',
 '[토막소식]강릉 양양 2021년 6월 2일',
 '[동정]강릉 양양 2021년 6월 2일',
 '[오늘의 날씨] 6월 1일',
 '[지천명의 오늘의 운세] 2021년 6월 1일 띠별운세',
 '[오늘 날씨] 6월 2일(수)',
 '미스터 달팽이 2021년 6월 2일자(이공명)',
 '[오늘의 운세] 2021년 6월 1일 화요일(음력 4월 21일/일진:경진(庚辰)/띠별·생년월일 운세)',
 '이틀 연속 감소세지만 재확산 가능성 여전…“6월, 중요 시기”',
 "전남농협, 6월 '이달의 새농민상' 담양·장흥 2부부 선정",
 '본사 사령 (6월 1일자)',
 '[오늘의 운세] 6월 1일 화요일 (음력 4월 21일 /庚辰) 띠별 / 생년월일 운세',
 '[날씨] 6월 2일(수) "맑음"',
 '6월 2일은 ‘유기농데이’…친환경농업협회, 기후변화 알리는 행사 개최',
 'ROTC 창설 제60주년 기념식, 6월 1일 개최',
 '배호 그림세상 (2021.06.02)',
 '6월1일(화) TV 편성표',
 '[오늘의 운세] 6월 1일 ( 음 4월 21일 )

In [20]:
# 주제어 추출 확인
from sklearn.feature_extraction.text import CountVectorizer

# tf-idf 값 구함
def c_tf_idf(documents, m, ngram_range=(1,1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1,1)
    tf_idf = np.multiply(tf, idf)
    
    return tf_idf, count

# tf_idf에서 n개의 topic 추출
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [59]:
# 카테고리별 클러스터별 주제어 추출
category_id = 3

tf_idf, count = c_tf_idf(topics[category_id].Doc.values, m=len(corpus))
top_n_words = extract_top_n_words_per_topic(tf_idf, count, topics[category_id], n=20)
topic_sizes = extract_topic_sizes(rslt[category_id])
topic_sizes.head(10)

,Topic,Size
0,-1,2272
25,24,83
6,5,72
8,7,57
80,79,46
5,4,45
10,9,43
34,33,41
117,116,39
71,70,38


In [60]:
print(f'len(topic_sizes):{len(topic_sizes)}')
print(topics[category_id])
print()

# 2번째 클러스터링 항목들에 대해 주제어 추출
idx = 2
print('---------------------------------------')
print(topics[category_id].Doc[idx:idx+1])  
print()
# 2번째 주제어일때 top_n_words idx=-1해줌.
print(top_n_words[idx-1][:10])

len(topic_sizes):125
     Topic                                                Doc
0       -1  생산성본부・가평군, 소상공인 경영환경개선 지원 나선다중기부, 지역가치 창업가 협업지...
1        0  "윤석열, '내 장모, 누구한테 피해 준 적 없다'"국민의힘 입당 저울질?…"장모,...
2        1  “아동학대 예방”… 현대차 ‘아이케어카’ 기부“아동학대 근절·안전한 보육”…‘은평형...
3        2  공공폐자원관리시설, 설치지역 주민과 운영이익 나눈다공공폐자원관리시설 설치 시 주민에...
4        3  검찰, 의료법 위반 혐의로 기소된 윤석열 장모 징역 3년 구형검찰, '요양급여 부정...
..     ...                                                ...
120    119  부산무용협회 회장에 김갑용 춤과사람대표김용인 재향경우회 제23대 중앙회장 공식 취임...
121    120  피의자 김오수 ‘김학의 사건’ 무혐의 될듯… 이광철은 대검차장이 기소여부 판단대검 ...
122    121  김오수 “굳건한 방파제 돼 정치적 중립 지킬 것”김오수 "방파제돼 부당한 압력으로부...
123    122  오늘부터 양도세 인상 '최고 75%'…김오수 검찰총장 취임  첫 출근하는 김오수 신...
124    123  문 대통령, 김오수 검찰총장 임명...“공정한 검찰 거듭나는데 큰 역할 해달라”김오...

[125 rows x 2 columns]

---------------------------------------
2    “아동학대 예방”… 현대차 ‘아이케어카’ 기부“아동학대 근절·안전한 보육”…‘은평형...
Name: Doc, dtype: object

[('아동학대', 0.34008910891394256), ('아이케어카', 0.31052564778152075), ('예방', 0.1823746723480524